# Beta Diversity Analysis, Rarefaction and Significance Tests

        

#### Notebook overview 

[1. Setup](#sus)<br>
[2. Metadata](#mdata)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.1 _Importing metadata in panda_](#import)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.2 _Selection of categorical variables of interest_](#selcatvar)<br>
[3. Visual inspection](#visui)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[3.1 _3D PCoA plots inspection_](#3d)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[3.2 _2D plot of main PCoA axis_](#2d)<br>
[4. Statistical analysis](#statistics)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[3.1 _3D PCoA plots inspection_](#permanova)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[3.2 _2D plot of main PCoA axis_](#adonis)<br>
[Note](#note)<br>


<a id='sus'></a>
## 1. Set_up

In [1]:
import os
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import qiime2 as q2
import seaborn as sns
from skbio import OrdinationResults
from qiime2 import Visualization
import matplotlib.pyplot as plt
from seaborn import scatterplot
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

%matplotlib inline

In [2]:
data_dir ='project_data'

In [1]:
def confidence_ellipse(x, y, ax, n_std=3.0, facecolor='none', **kwargs):
    """
    Create a plot of the covariance confidence ellipse of *x* and *y*.

    Parameters
    ----------
    x, y : array-like, shape (n, )
        Input data.

    ax : matplotlib.axes.Axes
        The axes object to draw the ellipse into.

    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.

    **kwargs
        Forwarded to `~matplotlib.patches.Ellipse`

    Returns
    -------
    matplotlib.patches.Ellipse
    """
    if x.size != y.size:
        raise ValueError("x and y must be the same size")

    cov = np.cov(x, y)
    pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensional dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0), width=ell_radius_x * 2, height=ell_radius_y * 2,
                      facecolor=facecolor, **kwargs)

    # Calculating the standard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = np.mean(x)

    # calculating the standard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = np.mean(y)

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)

In order to run this code, you need results of functions done on previous file : 
- `metadata` gathering is done in file A
- `diversity computation` is done in file F

<a id='sec1'></a>

<a id='mdata'></a>
## 2. Metadata

<a id='import'></a>
### 2.1. Importing metadata in panda

Metadata of refers to students additional informations we collected into the file "sample_meta_data.tsv".

In [14]:
df_meta = pd.read_csv(f'{data_dir}/sample_meta_data.tsv', sep='\t')

<a id='selcatvar'></a>
### 2.2. Selection of categorical variables of interest

In [23]:
df_cat = df_meta.select_dtypes(include=['object'])
df_cat.describe()

,GEN_age_cat,GEN_bmi_cat,GEN_collection_timestamp,GEN_country,GEN_geo_loc_name,GEN_host_common_name,GEN_last_move,GEN_last_travel,GEN_level_of_education,GEN_race,...,NUT_probiotic_frequency,NUT_red_meat_frequency,NUT_salted_snacks_frequency,NUT_seafood_frequency,NUT_sugary_sweets_frequency,NUT_vegetable_frequency,NUT_vitamin_b_supplement_frequency,NUT_vitamin_d_supplement_frequency,NUT_whole_eggs,NUT_whole_grain_frequency
count,523,523,523,523,523,523,523,523,523,523,...,523,523,523,523,523,523,523,523,523,523
unique,9,5,505,17,62,1,6,6,8,6,...,6,6,6,6,6,6,6,6,6,6
top,50s,Normal,2016-08-01 08:00:00,United Kingdom,United Kingdom:England,human,I have lived in my current state of residence ...,I have not been outside of my country of resid...,Graduate or Professional degree,Caucasian,...,Never,Occasionally,Rarely,Occasionally,Rarely,Daily,Never,Never,Occasionally,Regularly
freq,121,274,5,255,169,523,475,168,232,474,...,216,200,198,232,150,260,328,268,215,147


In [26]:
for col in df_cat.columns:
    print(col)

GEN_age_cat
GEN_bmi_cat
GEN_collection_timestamp
GEN_country
GEN_geo_loc_name
GEN_host_common_name
GEN_last_move
GEN_last_travel
GEN_level_of_education
GEN_race
GEN_sample_type
GEN_sex
NUT_alcohol_frequency
NUT_artificial_sweeteners
NUT_diet_type
NUT_drinks_per_session
NUT_fed_as_infant
NUT_fermented_plant_frequency
NUT_frozen_dessert_frequency
NUT_fruit_frequency
NUT_gluten
NUT_high_fat_red_meat_frequency
NUT_homecooked_meals_frequency
NUT_meat_eggs_frequency
NUT_milk_cheese_frequency
NUT_milk_substitute_frequency
NUT_olive_oil
NUT_poultry_frequency
NUT_prepared_meals_frequency
NUT_probiotic_frequency
NUT_red_meat_frequency
NUT_salted_snacks_frequency
NUT_seafood_frequency
NUT_sugary_sweets_frequency
NUT_vegetable_frequency
NUT_vitamin_b_supplement_frequency
NUT_vitamin_d_supplement_frequency
NUT_whole_eggs
NUT_whole_grain_frequency


In [7]:
md = q2.Metadata.load(data_dir + '/sample_meta_data.tsv').to_dataframe()
pd.DataFrame([str(sorted(md[col].astype(str).unique())) for col in md.columns],
             index=pd.Index(md.columns, name='Column'), columns=['Values'])

,Values
Column,
GEN_age_cat,"['20s', '30s', '40s', '50s', '60s', '70+', 'No..."
GEN_age_corrected,"['11.0', '14.0', '15.0', '16.0', '17.0', '18.0..."
GEN_bmi_cat,"['Normal', 'Not provided', 'Obese', 'Overweigh..."
GEN_bmi_corrected,"['11.2', '11.57', '14.34', '14.62', '14.79', '..."
GEN_cat,"['False', 'True']"
GEN_collection_timestamp,"['2014-05-10 12:00:00', '2015-01-01 09:00:00',..."
GEN_country,"['Australia', 'Belgium', 'Canada', 'Georgia', ..."
GEN_dog,"['False', 'True']"
GEN_elevation,"['-0.9', '0.0', '1.4', '10.4', '10.8', '10.9',..."


Almost all the variables present in the dataframe df_cat can be interesting to test for beta-analysis. However, for the purpose of our research goals here, we will focus on data about age, BMI, country, level of education

<a id='visui'></a>
## 3. Visual inspection

Beta diversity measures the similarity between samples or groups of samples.        
To inspect groupings of beta diversity metrics across metadata categories, we will start by inspecting the principal coordinates (PCoA) plots created with the `qiime diversity core-metrics-phylogenetic` method in file F (directory core-metrics-results)

<a id='3d'></a>
### 3.1. 3D PCoA plots inspection

In [ ]:
Visualization.load(f'{data_dir}/core-metrics-results/weighted_unifrac_emperor.qzv')

In [106]:
Visualization.load(f'{data_dir}/core-metrics-results/bray_curtis_emperor.qzv')

<visualization: Visualization uuid: 6af57913-dd4b-427b-a22e-4aaaeb32c554>

<a id='2d'></a>
### 3.2. 2D plot of main PCoA axis

In [22]:
pcs = q2.Artifact.load(os.path.join(data_dir, 'core-metrics-results/bray_curtis_pcoa_results.qza'))
pcs = pcs.view(OrdinationResults)
pcs_data = pcs.samples.iloc[:,:3]
pcs_data.columns = ['Axis 1', 'Axis 2', 'Axis 3']

In [15]:
pcs_data.head()

,Axis 1,Axis 2,Axis 3
10317.000002929,-0.144197,-0.023014,0.022661
10317.000002930,-0.139421,-0.016882,0.178423
10317.000003283,0.026313,0.051591,0.002213
10317.000004079,-0.150133,0.030451,-0.045630
10317.000016169,-0.069590,0.008753,-0.035616


In [23]:
pcs_data_with_md = pcs_data.join(md['NUT_vegetable_frequency'])

In [2]:
selNe= pcs_data_with_md.loc[pcs_data_with_md['NUT_vegetable_frequency']=='Never']
selRa= pcs_data_with_md.loc[pcs_data_with_md['NUT_vegetable_frequency']=='Rarely']
selO= pcs_data_with_md.loc[pcs_data_with_md['NUT_vegetable_frequency']=='Occasionally']
selRe= pcs_data_with_md.loc[pcs_data_with_md['NUT_vegetable_frequency']=='Regularly']
selD= pcs_data_with_md.loc[pcs_data_with_md['NUT_vegetable_frequency']=='Daily']
selNp= pcs_data_with_md.loc[pcs_data_with_md['NUT_vegetable_frequency']=='Not provided']

pcs_data_with_md['NUT_vegetable_frequency'] = pd.Categorical(pcs_data_with_md['NUT_vegetable_frequency'], 
                      categories=["Never", "Rarely", "Occasionally", "Regularly", "Daily", 'Not provided'],
                      ordered=True)


with sns.axes_style("whitegrid"):
    fig, ax = plt.subplots()
    sns.set(rc={'figure.figsize':(10,10)}, style='white')
    sns.scatterplot(data=pcs_data_with_md, x='Axis 1', y='Axis 2', ax=ax, hue=pcs_data_with_md['NUT_vegetable_frequency'],palette='rocket')
        
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=14, title_fontsize=14)
    legend = ax.get_legend()
    legend.set_title('NUT_vegetable_frequency')
    

    ax.tick_params(axis='x', which='major', bottom=True) 
    ax.tick_params(axis='y', which='major', left=True)

    confidence_ellipse(selRe['Axis 2'], selRe['Axis 1'], ax, edgecolor='red')
    confidence_ellipse(selNp['Axis 2'], selNp['Axis 1'], ax, edgecolor='lightsalmon')
    confidence_ellipse(selD['Axis 2'], selD['Axis 1'], ax, edgecolor='orangered')
    confidence_ellipse(selNe['Axis 2'], selNe['Axis 1'], ax, edgecolor='black')
    confidence_ellipse(selO['Axis 2'], selO['Axis 1'], ax, edgecolor='mediumvioletred')
    confidence_ellipse(selRa['Axis 2'], selRa['Axis 1'], ax, edgecolor='purple')
    
    
    plt.savefig('spveg2.png',bbox_inches='tight', dpi=300)


NameError: name 'pcs_data_with_md' is not defined

<a id='statistics'></a>
## 4.Statistical analysis

<a id='permanova'></a>
### 4.1. Non-parametric multivariate analysis of variance : PERMANOVA testing of variable

Associations between beta diversity and categorical variables can be statistically tested using a PERMANOVA test. This is a non-parametric statistical test that checks the null hypothesis that the distances between samples of one group are equivalent to distances to samples of another group. If this null hypothesis is rejected, we can infer that the distances between samples of one group differ significantly from the distances to samples in at least one other group. We can perform a PERMANOVA test checking whether the observed categories are significantly grouped in QIIME 2 with the `qiime diversity beta-group-significance` method: 

In [40]:
os.mkdir('/home/jovyan/HealthyFood/JupyterDocs/project_data/core-metrics-results-bd')

**a)** Example : variable "NUT_prepared_meals_frequency"

##### *with  weighted UniFrac distance matrix*

In [124]:
! qiime diversity beta-group-significance \
    --i-distance-matrix $data_dir/core-metrics-results/weighted_unifrac_distance_matrix.qza \
    --m-metadata-file $data_dir/sample_meta_data.tsv \
    --m-metadata-column NUT_prepared_meals_frequency \
--p-pairwise \
--o-visualization $data_dir/core-metrics-results-bd/wu_pmf-significance.qzv

Saved Visualization to: project_data/core-metrics-results-bd/wu_pmf-significance.qzv


In [4]:
Visualization.load(f'{data_dir}/core-metrics-results-bd/wu_vf-significance.qzv')

<visualization: Visualization uuid: da387cb6-676c-4440-be22-8f5b7d1d98a6>

##### *with Bray curtis distance matrix*

In [104]:
! qiime diversity beta-group-significance \
    --i-distance-matrix $data_dir/core-metrics-results/bray_curtis_distance_matrix.qza \
    --m-metadata-file $data_dir/sample_meta_data.tsv \
    --m-metadata-column GEN_sex\
--p-pairwise \
--o-visualization $data_dir/core-metrics-results-bd/bc_sex-significance.qzv

Saved Visualization to: project_data/core-metrics-results-bd/bc_sex-significance.qzv


In [105]:
Visualization.load(f'{data_dir}/core-metrics-results-bd/bc_sex-significance.qzv')

<visualization: Visualization uuid: 007a40d5-82a6-486f-90ef-50a84944be9d>

<a id='adonis'></a>
### 4.2. Adonis implementation of PERMANOVA tests : Multfactor testing

The `adonis` implementation of PERMANOVA (part of the r-vegan package) accepts a formula as input, which can consist of one or more independent terms. This might be useful for testing which covariates explain the most variation in our datasets.

##### Hypothesis 1 : Milk products 

In [33]:
# testing with bray curtis matrix
! qiime diversity adonis \
    --i-distance-matrix $data_dir/core-metrics-results/bray_curtis_distance_matrix.qza \
    --m-metadata-file $data_dir/sample_meta_data.tsv \
    --p-formula "NUT_milk_cheese_frequency*NUT_milk_substitute_frequency*NUT_vitamin_d_supplement_frequency" \
    --o-visualization $data_dir/core-metrics-results-bd/adonis-bc-H1.qzv
Visualization.load(f'{data_dir}/core-metrics-results-bd/adonis-bc-H1.qzv')

Saved Visualization to: project_data/core-metrics-results-bd/adonis-bc-H1.qzv


<visualization: Visualization uuid: 7e727ef9-e5e2-41e4-a4eb-dbbb5fdf47c4>

In [19]:
# testing with weighted UniFrac matrix
! qiime diversity adonis \
    --i-distance-matrix $data_dir/core-metrics-results/weighted_unifrac_distance_matrix.qza \
    --m-metadata-file $data_dir/sample_meta_data.tsv \
    --p-formula "NUT_milk_cheese_frequency*NUT_milk_substitute_frequency*NUT_vitamin_d_supplement_frequency" \
    --o-visualization $data_dir/core-metrics-results-bd/adonis-wu-H1.qzv
Visualization.load(f'{data_dir}/core-metrics-results-bd/adonis-wu-H1.qzv')

Saved Visualization to: project_data/core-metrics-results-bd/adonis-wu-H1.qzv


<visualization: Visualization uuid: afff41c9-9e23-48ca-a2fa-02e0887b7f6e>

#####  Hypothesis 2 : Mediterranean diet

In [36]:
# testing with bray curtis matrix
! qiime diversity adonis \
    --i-distance-matrix $data_dir/core-metrics-results/bray_curtis_distance_matrix.qza \
    --m-metadata-file $data_dir/sample_meta_data.tsv \
    --p-formula "NUT_olive_oil*NUT_seafood_frequency*NUT_vegetable_frequency*NUT_fruit_frequency*NUT_whole_grain_frequency" \
    --o-visualization $data_dir/core-metrics-results-bd/adonis-bc-H2.qzv
Visualization.load(f'{data_dir}/core-metrics-results-bd/adonis-bc-H2.qzv')

^C


<visualization: Visualization uuid: 9caf9b29-995e-478c-a873-332d86e66856>

In [21]:
# testing with weighted unifrac matrix
! qiime diversity adonis \
    --i-distance-matrix $data_dir/core-metrics-results/weighted_unifrac_distance_matrix.qza \
    --m-metadata-file $data_dir/sample_meta_data.tsv \
    --p-formula "NUT_olive_oil*NUT_seafood_frequency*NUT_vegetable_frequency*NUT_fruit_frequency*NUT_whole_grain_frequency" \
    --o-visualization $data_dir/core-metrics-results-bd/adonis-wu-H2.qzv
Visualization.load(f'{data_dir}/core-metrics-results-bd/adonis-wu-H2.qzv')

Saved Visualization to: project_data/core-metrics-results-bd/adonis-wu-H2.qzv


<visualization: Visualization uuid: 6c98d89e-09b1-426d-aa92-554aa94fe40b>

#####  Hypothesis 3 : Poultry and Meat

In [6]:
# testing with bray curtis matrix
! qiime diversity adonis \
    --i-distance-matrix $data_dir/core-metrics-results/bray_curtis_distance_matrix.qza \
    --m-metadata-file $data_dir/sample_meta_data.tsv \
    --p-formula "NUT_whole_eggs*NUT_poultry_frequency*NUT_high_fat_red_meat_frequency*NUT_meat_eggs_frequency*NUT_red_meat_frequency*NUT_vitamin_b_supplement_frequency" \
    --o-visualization $data_dir/core-metrics-results-bd/adonis-bc-H3.qzv
Visualization.load(f'{data_dir}/core-metrics-results-bd/adonis-bc-H3.qzv')

^C

Aborted!


ValueError: project_data/core-metrics-results-bd/adonis-bc-H3.qzv does not exist.

In [ ]:
# testing with weighted unifrac matrix
! qiime diversity adonis \
    --i-distance-matrix $data_dir/core-metrics-results/weighted_unifrac_distance_matrix.qza \
    --m-metadata-file $data_dir/sample_meta_data.tsv \
    --p-formula "NUT_whole_eggs*NUT_poultry_frequency*NUT_high_fat_red_meat_frequency*NUT_meat_eggs_frequency*NUT_red_meat_frequency*NUT_vitamin_b_supplement_frequency" \
    --o-visualization $data_dir/core-metrics-results-bd/adonis-wu-H3.qzv
Visualization.load(f'{data_dir}/core-metrics-results-bd/adonis-wu-H3.qzv')

In [4]:
# testing with bray curtis matrix
! qiime diversity adonis \
    --i-distance-matrix $data_dir/core-metrics-results/bray_curtis_distance_matrix.qza \
    --m-metadata-file $data_dir/sample_meta_data.tsv \
    --p-formula "NUT_alcohol_frequency*NUT_drinks_per_session" \
    --o-visualization $data_dir/core-metrics-results-bd/adonis-bc-H4.qzv
Visualization.load(f'{data_dir}/core-metrics-results-bd/adonis-bc-H4.qzv')

Saved Visualization to: project_data/core-metrics-results-bd/adonis-bc-H4.qzv


<visualization: Visualization uuid: f55a2cd1-7432-4b44-a4a3-fd181a5f97c0>

In [5]:
# testing with weighted unifrac matrix
! qiime diversity adonis \
    --i-distance-matrix $data_dir/core-metrics-results/weighted_unifrac_distance_matrix.qza \
    --m-metadata-file $data_dir/sample_meta_data.tsv \
    --p-formula "NUT_alcohol_frequency*NUT_drinks_per_session" \
    --o-visualization $data_dir/core-metrics-results-bd/adonis-wu-H4.qzv
Visualization.load(f'{data_dir}/core-metrics-results-bd/adonis-wu-H4.qzv')

Saved Visualization to: project_data/core-metrics-results-bd/adonis-wu-H4.qzv


<visualization: Visualization uuid: c037b3c9-e92c-462b-803a-2483529b4773>